In [1]:
%load_ext autoreload
%autoreload 2

import os

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter
from generator.preprocess import *
from generator.preprocess import remove_outlier_trajectories
from generator.road_network import RoadNetwork
import glob
from tqdm import tqdm
import torch

In [6]:
"""
 Generate osmnx network from csv
"""

df = pd.read_csv("../datasets/trajectories/hanover/temporal/hannover_streetgraph.csv")
df["geometry"] = df["geometry"].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry="geometry")
gdf["coords"] = gdf["geometry"].swifter.apply(lambda x: list(x.coords))

Pandas Apply:   0%|          | 0/8620 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/8620 [00:00<?, ?it/s]

In [6]:
gdf = ox.io._stringify_nonnumeric_cols(gdf)
gdf["fid"] = np.arange(0, gdf.shape[0], dtype="int")  # id for each edge
gdf.to_file("../osm_data/hanover_temp" + "/edges.shp", encoding="utf-8")

In [7]:
gdf

,Unnamed: 0,source,target,geometry,gm_id,id,fid,speed_limit,coords,length,osm_ids,type
0,0,40315,40316,"LINESTRING (9.67027 52.35957, 9.67032 52.35933)",087c3c0e-237e-0b4e-263b-7c08b0234e0b,24631,0,30.0,"[(9.67027, 52.3595659), (9.6703202, 52.3593341...",25.999437,"'{1862717287,250566185}'",tertiary
1,1,40315,85185,"LINESTRING (9.67027 52.35957, 9.67009 52.36072)",087c3c0e-237e-0b4e-9440-7c08c5224e0b,92181,1,50.0,"[(9.67027, 52.3595659), (9.6700853, 52.3607237)]",129.350998,"'{1862717287,251031226}'",tertiary
2,2,40316,55012,"LINESTRING (9.67032 52.35933, 9.67036 52.35923)",087c3b26-23b0-0b4e-bc3a-7c08d6234e0b,37452,2,30.0,"[(9.6703202, 52.359334100000005), (9.670358400...",12.025311,"'{250566185,929530747}'",tertiary
3,3,40316,40315,"LINESTRING (9.67032 52.35933, 9.67027 52.35957)",087c3b26-23b0-0b4e-0e3c-7c087e234e0b,121667,3,30.0,"[(9.6703202, 52.359334100000005), (9.67027, 52...",25.999437,"'{250566185,1862717287}'",tertiary
4,4,66994,66082,"LINESTRING (9.68702 52.39259, 9.68686 52.39265)",087cbd0e-64ea-0b4e-4fbd-7c0848644e0b,53635,4,50.0,"[(9.6870185, 52.3925898), (9.686855900000001, ...",13.177832,"'{269501492,402880222}'",tertiary
...,...,...,...,...,...,...,...,...,...,...,...,...
8615,8615,96374,93768,"LINESTRING (9.66935 52.34770, 9.66842 52.34766...",087c0db3-1fe9-0b4e-fa0e-7c0851034e0b,178804,8615,100.0,"[(9.6693532, 52.3476987), (9.668417400000001, ...",502.058533,"'{248208935,1677375037,31115703,1677375034,252...",trunk
8616,8616,96381,23053,"LINESTRING (9.86166 52.35802, 9.86153 52.35798...",087c3602-0f1c-0b51-ad34-7c086d0c510b,177020,8616,50.0,"[(9.8616597, 52.358017600000004), (9.8615315, ...",60.601940,"'{2220038967,1471968252,314682062,1678035710,2...",trunk_link
8617,8617,96383,86769,"LINESTRING (9.86458 52.33736, 9.86456 52.33713...",087be54c-1a82-0b51-80e0-7b083b1a510b,177536,8617,60.0,"[(9.864577500000001, 52.3373562), (9.8645636, ...",136.677246,"'{430093532,2119966918,510765,2119966915,493121}'",motorway_link
8618,8618,96392,34036,"LINESTRING (9.70888 52.40846, 9.70919 52.40810...",087cfb0f-ba51-0b4e-2af9-7c08f0bb4e0b,180333,8618,50.0,"[(9.7088807, 52.408463100000006), (9.709193200...",60.899536,"'{322022894,2904928815,322022768}'",secondary


In [4]:
G = nx.from_pandas_edgelist(gdf, "source", "target", True, nx.MultiDiGraph)
sG = [G.subgraph(c) for c in sorted(nx.weakly_connected_components(G), key=len, reverse=True)][0]
sdf = nx.to_pandas_edgelist(sG)
sdf["coords"] = sdf["geometry"].swifter.apply(lambda x: list(x.coords))
sdf.to_csv("../datasets/trajectories/hanover/temporal/hannover_streetgraph.csv")

Pandas Apply:   0%|          | 0/8620 [00:00<?, ?it/s]

In [ ]:
# Folium Heat Map
from folium import plugins
from folium.plugins import HeatMap
from collections import defaultdict
import branca.colormap

m = folium.Map(location=[52.37052, 9.73322],
                    zoom_start = 8)

coords = gdf.iloc[:, -1].values

for line in coords:
    data = [(c[1], c[0]) for c in line]
    folium.PolyLine(data, color="red", weight=2.5, opacity=0.8).add_to(m)

coords = sdf["coords"].values
for line in coords:
    data = [(c[1], c[0]) for c in line]
    folium.PolyLine(data, color="green", weight=2.5, opacity=0.8).add_to(m)

# Display the map
#map_porto.save("heatmap_gps_points_porto.html")
m

In [15]:
network = RoadNetwork()
network.load_hanover_temporal(path="../datasets/trajectories/hanover/temporal/hannover_streetgraph.csv")

Pandas Apply:   0%|          | 0/8620 [00:00<?, ?it/s]

In [16]:
network.gdf_edges

Unnamed: 0  \
u     v     key               
40315 40316 0             0   
      85185 0             1   
40316 55012 0             2   
      40315 0             3   
66994 66082 0             4   
...                     ...   
96374 93768 0          8615   
96381 23053 0          8616   
96383 86769 0          8617   
96392 34036 0          8618   
96394 96359 0          8619   

                                                          geometry  \
u     v     key                                                      
40315 40316 0      LINESTRING (9.67027 52.35957, 9.67032 52.35933)   
      85185 0      LINESTRING (9.67027 52.35957, 9.67009 52.36072)   
40316 55012 0      LINESTRING (9.67032 52.35933, 9.67036 52.35923)   
      40315 0      LINESTRING (9.67032 52.35933, 9.67027 52.35957)   
66994 66082 0      LINESTRING (9.68702 52.39259, 9.68686 52.39265)   
...                                                            ...   
96374 93768 0    LINESTRING (9.66935 52.34770, 9.66842 52.34766...   
96381 23053 0    LINESTRING (9.86166 52.35802, 9.86153 52.35798...   
96383 86769 0    LINESTRING (9.86458 52.33736, 9.86456 52.33713...   
96392 34036 0    LINESTRING (9.70888 52.40846, 9.70919 52.40810...   
96394 96359 0      LINESTRING (9.72290 52.44195, 9.72294 52.44192)   

                                                gm_id      id   fid  \
u     v     key                                                       
40315 40316 0    087c3c0e-237e-0b4e-263b-7c08b0234e0b   24631     0   
      85185 0    087c3c0e-237e-0b4e-9440-7c08c5224e0b   92181     1   
40316 55012 0    087c3b26-23b0-0b4e-bc3a-7c08d6234e0b   37452     2   
      40315 0    087c3b26-23b0-0b4e-0e3c-7c087e234e0b  121667     3   
66994 66082 0    087cbd0e-64ea-0b4e-4fbd-7c0848644e0b   53635     4   
...                                               ...     ...   ...   
96374 93768 0    087c0db3-1fe9-0b4e-fa0e-7c0851034e0b  178804  8615   
96381 23053 0    087c3602-0f1c-0b51-ad34-7c086d0c510b  177020  8616   
96383 86769 0    087be54c-1a82-0b51-80e0-7b083b1a510b  177536  8617   
96392 34036 0    087cfb0f-ba51-0b4e-2af9-7c08f0bb4e0b  180333  8618   
96394 96359 0    087d7de3-f113-0b4e-bd7d-7d083af14e0b  181391  8619   

                 speed_limit  \
u     v     key                
40315 40316 0           30.0   
      85185 0           50.0   
40316 55012 0           30.0   
      40315 0           30.0   
66994 66082 0           50.0   
...                      ...   
96374 93768 0          100.0   
96381 23053 0           50.0   
96383 86769 0           60.0   
96392 34036 0           50.0   
96394 96359 0           70.0   

                                                            coords  \
u     v     key                                                      
40315 40316 0    [(9.67027, 52.3595659), (9.6703202, 52.3593341...   
      85185 0     [(9.67027, 52.3595659), (9.6700853, 52.3607237)]   
40316 55012 0    [(9.6703202, 52.359334100000005), (9.670358400...   
      40315 0    [(9.6703202, 52.359334100000005), (9.67027, 52...   
66994 66082 0    [(9.6870185, 52.3925898), (9.686855900000001, ...   
...                                                            ...   
96374 93768 0    [(9.6693532, 52.3476987), (9.668417400000001, ...   
96381 23053 0    [(9.8616597, 52.358017600000004), (9.8615315, ...   
96383 86769 0    [(9.864577500000001, 52.3373562), (9.8645636, ...   
96392 34036 0    [(9.7088807, 52.408463100000006), (9.709193200...   
96394 96359 0    [(9.722899400000001, 52.4419545), (9.7229378, ...   

                     length  \
u     v     key               
40315 40316 0     25.999437   
      85185 0    129.350998   
40316 55012 0     12.025311   
      40315 0     25.999437   
66994 66082 0     13.177832   
...                     ...   
96374 93768 0    502.058533   
96381 23053 0     60.601940   
96383 86769 0    136.677246   
96392 34036 0     60.899536   
96394 96359 0      4.915459   

                                              

In [ ]:
""" 
Speed data analysis
"""

In [13]:
temporal = pd.read_csv("../datasets/trajectories/hanover/temporal/hannover_traffic.csv")

In [17]:
# General temporal data (Note: Nodes are ordered by line graph)
temporal["time"] = pd.to_datetime(temporal["time"])
max_steps, min_steps = temporal["time"].max(), temporal["time"].min()
pad = 0
data = []
for i, index in tqdm(enumerate(network.line_graph.nodes)):
    row = network.gdf_edges.loc[index]
    temp = temporal[temporal["id"]==row["id"]][["time", "speed"]].sort_values("time")
    temp = temp.set_index('time')
    if min_steps not in temp.index:
        temp.loc[min_steps] = pad
    if max_steps not in temp.index:
        temp.loc[max_steps] = pad
    temp = temp.asfreq('15Min', fill_value=0)
    temp[["length", "speed_limit", "highway_enc"]] = row[["length", "speed_limit", "highway_enc"]]
    data.append(temp.values)

8620it [07:28, 19.24it/s]


In [18]:
x = torch.Tensor(data)
x.shape

torch.Size([8620, 5952, 4])

In [19]:
x.isnan().sum()

tensor(0)

In [20]:
torch.save(x, 'temporal_data.pt')

In [12]:
# map trajectories to network
df = pd.read_csv("../datasets/trajectories/hanover/mapped_id_poly_clipped.csv", sep=";")
df

,timestamp,POLYLINE,id
0,0 80 424 535 613 1094 1171 1453 1528 1...,"LINESTRING (9.8288 52.3326, 9.819705 52.331356...",1
1,0 30 60 90 120 172 202 232 262 292 545 57...,"LINESTRING (9.660448261338471 52.3136182, 9.66...",2
2,0 90 180 229 319 409 492 582 672 762,"LINESTRING (9.7119 52.343, 9.716048 52.344969,...",3
3,0 79 141 224 377 453 580 685 763 ...,"LINESTRING (9.7585 52.3363, 9.762929 52.34235,...",4
4,0 1 9 15 26 30 34 41 46 ...,"LINESTRING (9.6138059 52.428522640720765, 9.62...",5
...,...,...,...
143854,0 5 8 ... 6035 6038 6043,"LINESTRING (9.718658264790285 52.4474529, 9.71...",143855
143855,0 5 8 16 19 20 25 29 35 ...,"LINESTRING (9.7278 52.3654, 9.727219 52.365731...",143856
143856,0 599 703 718 733 748 763 778 1295 1...,"LINESTRING (9.782460425078042 52.4474529, 9.78...",143857
143857,0 5 10 15 20 25 30 35 45 ...,"LINESTRING (9.738963 52.382663, 9.738996 52.38...",143858


In [8]:
network.fmm_trajectorie_mapping(
    network_file="../osm_data/hanover_temp/edges.shp",
    input_file="../datasets/trajectories/hanover/mapped_id_poly_clipped.csv",
    output_file="../datasets/trajectories/hanover/temporal/road-segment-mapping.txt"
    source_field="source",
    target_field="target"
)

[2022-09-09 02:05:33.476] [info] [network.cpp:72] Read network from file ../osm_data/hanover_temp/edges.shp
[2022-09-09 02:05:33.476] [critical] [network.cpp:97] Field not found: fid index 10, u index -1, v index -1


RuntimeError: Field not found: fid index 10, u index -1, v index -1